
# 문제 4-1 : 카페 메뉴 도구(Tool) 호출 체인 구현 (LangChain)
이 노트북은 LangChain의 **Tool Calling** 기능을 사용해, 로컬 벡터 DB/웹/위키피디아를 조회하는 카페 도움말 어시스턴트를 구현합니다.

## 준비사항
- `./data/cafe_menu.txt` (이 노트북에서 자동 생성 가능)
- 환경변수: `OPENAI_API_KEY`, `TAVILY_API_KEY`
- 벡터 DB: FAISS (`./db/cafe_db`에 저장)


## 1. 카페 메뉴 데이터 파일 생성

In [6]:

from pathlib import Path

DATA_DIR = Path.cwd() / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)
menu_path = DATA_DIR / "cafe_menu.txt"

if not menu_path.exists():
    menu_path.write_text(encoding="utf-8")
    print("Created:", menu_path)
else:
    print("Exists:", menu_path)
print(menu_path.read_text(encoding="utf-8")[:300], "...")


Exists: c:\mylangchain\mylangchain-app\src\mylangchain_app\0example\data\cafe_menu.txt

아메리카노
가격: ₩4,500
재료: 에스프레소, 뜨거운 물
설명: 원두 본연의 맛을 깔끔하게 즐길 수 있는 가장 기본 커피.

카페라떼
가격: ₩5,000
재료: 에스프레소, 스팀밀크, 우유거품
설명: 부드러운 우유 풍미와 조화로운 밸런스의 라떼.

바닐라라떼
가격: ₩5,500
재료: 에스프레소, 바닐라시럽, 스팀밀크
설명: 달콤한 바닐라 향이 매력적인 라떼.

카푸치노
가격: ₩5,200
재료: 에스프레소, 스팀밀크, 두꺼운 우유거품
설명: 진한 커피향과 거품 식감이 특징인 클래식 메뉴.

콜드브루
가격: ₩5,300
재료: ...


## 1-2. 벡터 DB 구축

In [2]:

import os, re, json
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from pathlib import Path

DB_DIR = Path.cwd() / "db" / "cafe_db"
DB_DIR.mkdir(parents=True, exist_ok=True)

txt = menu_path.read_text(encoding="utf-8")
blocks = [b.strip() for b in re.split(r"\n\s*\n", txt) if b.strip()]

docs = []
for block in blocks:
    lines = block.splitlines()
    menu_name = lines[0].strip()
    docs.append(Document(page_content=block, metadata={"menu_name": menu_name}))

emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(docs, emb)
vs.save_local(str(DB_DIR))
print("FAISS index saved to:", DB_DIR)


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS index saved to: c:\mylangchain\mylangchain-app\src\mylangchain_app\0example\db\cafe_db


## 2. 3개의 도구를 정의

In [3]:

import json
import wikipedia
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

DB_DIR = Path.cwd() / "db" / "cafe_db"
emb = OpenAIEmbeddings(model="text-embedding-3-small")

@tool("tavily_search_func", return_direct=False)
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다 (Tavily). 입력: 검색어(str). 출력: 요약 문자열."""
    tavily = TavilySearchResults(max_results=5)
    results = tavily.invoke({"query": query})
    out = []
    for i, r in enumerate(results, 1):
        out.append(f"[{i}] {r.get('url','')}\n{r.get('content','')[:300]}...")
    return "\n\n".join(out) if out else "검색 결과가 없습니다."

@tool("wiki_summary", return_direct=False)
def wiki_summary(topic: str) -> str:
    """위키피디아에서 주제 요약을 제공합니다. 입력: 주제(str). 출력: 요약 문자열."""
    try:
        wikipedia.set_lang("ko")
        return wikipedia.summary(topic, sentences=3, auto_suggest=False, redirect=True)
    except Exception as e:
        return f"위키 요약 실패: {e}"

@tool("db_search_cafe_func", return_direct=False)
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 유사한 항목을 검색합니다. 입력: 쿼리(str). 출력: JSON 문자열(List[Document])."""
    local_vs = FAISS.load_local(str(DB_DIR), embeddings=emb, allow_dangerous_deserialization=True)
    found = local_vs.similarity_search(query, k=4)
    payload = [{"page_content": d.page_content, "metadata": d.metadata} for d in found]
    return json.dumps(payload, ensure_ascii=False)


## 2-1. LLM 바인딩 및 체인
## 3. 간단한 도구 호출 체인 구현체인 구조

In [4]:

from typing import Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
from langchain_core.messages import AIMessage

TOOLS = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(TOOLS)

@chain
def cafe_tool_chain(question: str) -> Dict[str, Any]:
    ai: AIMessage = llm_with_tools.invoke(question)
    calls = getattr(ai, "tool_calls", []) or []
    tool_results = []

    if calls:
        for c in calls:
            name = c["name"]
            args = c.get("args", {})
            if name == "tavily_search_func":
                out = tavily_search_func.invoke(args.get("query", question))
            elif name == "wiki_summary":
                out = wiki_summary.invoke(args.get("topic", question))
            elif name == "db_search_cafe_func":
                out = db_search_cafe_func.invoke(args.get("query", question))
            else:
                out = f"알 수 없는 도구: {name}"
            tool_results.append({"tool": name, "output": out})

        ctx = "\n\n".join([f"[{r['tool']}]\n{r['output']}" for r in tool_results])
        final = llm.invoke(f"""사용자 질문: {question}
아래 도구 결과를 참고하여 한국어로 간결하고 정확하게 답하세요.

도구 결과:
{ctx}
""")
        return {"answer": final.content, "tool_calls": calls, "tool_results": tool_results}
    else:
        return {"answer": ai.content, "tool_calls": [], "tool_results": []}


## 4. 테스트 질문 처리

In [5]:

out = cafe_tool_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")
print("=== 최종 답변 ===\n", out["answer"])
print("\n=== 도구 호출 내역 ===\n", out["tool_calls"])
print("\n=== 도구 실행 결과(요약) ===")
for r in out["tool_results"]:
    print(f"- {r['tool']}: {str(r['output'])[:180]}...")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


=== 최종 답변 ===
 아메리카노의 가격은 ₩4,500이며, 에스프레소와 뜨거운 물로 만들어집니다. 이 커피는 원두 본연의 맛을 깔끔하게 즐길 수 있는 가장 기본적인 메뉴입니다.

=== 도구 호출 내역 ===
 [{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_OP3D2nbXVzFBUUWG3DbufM0r', 'type': 'tool_call'}, {'name': 'wiki_summary', 'args': {'topic': '아메리카노'}, 'id': 'call_pesWomEEZPj64mKru47qySNU', 'type': 'tool_call'}]

=== 도구 실행 결과(요약) ===
- db_search_cafe_func: [{"page_content": "아메리카노\n가격: ₩4,500\n재료: 에스프레소, 뜨거운 물\n설명: 원두 본연의 맛을 깔끔하게 즐길 수 있는 가장 기본 커피.", "metadata": {"menu_name": "아메리카노"}}, {"page_content": "카푸치노\n가격: ₩5,200\n재료: 에스프레소, 스...
- wiki_summary: 위키 요약 실패: "아메리카노" may refer to: 
카페 아메리카노
아메리카노
10cm
Americano (노래)
사비에르 쿠가트
아메리카노 (2005년 영화)
아메리카노 (2011년 영화)
아메리카누 FC
제목에 "아메리카노" 항목을 포함한 모든 문서
아메리카나
아메리칸...
